In [ ]:
import plotly
from algo import *
import pandas as pd

In [ ]:
import datetime

In [ ]:
def change_HMS(t=0):
    '''change int to h-m-s format 
    '''
    if t < 0 or t > 999999:
        return None, None, None
    
    h, m, s= int(t/10000), int(t%10000/100), int(t%100)

    return datetime.time(h, m, s)

In [ ]:
def change_to_HMS(t=''):
    '''change time format to int
    '''
    return int(str(int(t.hour))+"{:02d}".format(int(t.minute))+"{:02d}".format((int(t.second))))


In [ ]:
def check_cross(line1, line2, m = 5):
	if len(line1) < m or len(line2) < m:
		raise ValueError(f"Both line1 and line2 should have at least {m} values.")

	for i in range(len(line1) - m + 1):
		subset_1 = line1[i:i+m]
		subset_2 = line2[i:i+m]

		# 查找初始金叉
		if subset_1[0] < subset_2[0] and subset_1[1] > subset_2[1]:
			trend = "gold"
			# 检查是否K始终大于D
			if all(k > d for k, d in zip(subset_1[1:], subset_2[1:])):
				return i+1, trend

		# 查找初始死叉
		elif subset_1[0] > subset_2[0] and subset_1[1] < subset_2[1]:
			trend = "dead"
			# 检查是否K始终小于D
			if all(k < d for k, d in zip(subset_1[1:], subset_2[1:])):
				return i+1, trend

	return None, None


In [ ]:
def get_main_future_codes(end_date_string, threshold = 0.05):

	end_date = datetime.datetime.strptime(end_date_string, "%Y-%m-%d")
	end_date_ahead = end_date - timedelta(days=1)
	end_date_ahead_string = end_date_ahead.strftime('%Y-%m-%d')  # 格式为 YYYY-MM-DD

	if is_valid_date(end_date_string):

		"""
			C1: 正基差过滤
		"""
		t1 = time.time()
		# get namelist by basis of percentage
		print(end_date_ahead_string)
		basis_future = pd.DataFrame([i for i in DCE_commodity_price_collection.find(
						# {"date": end_date_string}
						{"date": end_date_ahead_string}
					).sort("main_future_code", pymongo.ASCENDING) ])
		# print(basis_future)
		# print(basis_future)
		basis_future = basis_future.sort_values(by='main_cf_basis_percent', ascending=False)

		t2 = time.time()

		# Find the main contract with +5% positive basis 
		codes = basis_future[basis_future["main_cf_basis_percent"]>=threshold]["main_future_code"].tolist()
		codes = list(set(codes))
		return codes

In [ ]:
main_future_codes = get_main_future_codes('2023-09-13')
print(main_future_codes)

In [ ]:
def get_main_future_price_period(codes,tt=[]):
    pass

In [ ]:
import pprint
def get_realtime_price(date, tt, codelist=[]):
    realtime_prices_df = pd.DataFrame()
    # print(codelist)
    for code in codelist:
        cur_realtime_prices = DCE_future_price_collection.find(
            {"$and":[
                {'date':date},
                {'clock':{"$lt":tt}},
                {'future_code':code}]}
            ).sort("clock", pymongo.DESCENDING).limit(1)
        cur_df = pd.DataFrame(list(cur_realtime_prices))
        # print(f'xxxxx')
        # pprint.pprint(cur_df.loc[0,:])
        # print(f'xxxxx')
        realtime_prices_df = pd.concat([realtime_prices_df, cur_df], ignore_index=True)

    return realtime_prices_df

In [ ]:
def quick_cursor(code, end_date_string,df, min_count = 1500, retry = 1):
	if retry >0 and len(df) < min_count:
		t1 = time.time()

		minutes_cursors = DCE_future_price_collection.find(
						{"$and":[
							{'date':end_date_string},
							{'future_code': code}]}
						).sort("clock", pymongo.DESCENDING)

		cur_df = pd.DataFrame(list(minutes_cursors))
		t2 = time.time()

		if len(cur_df)> 0:
			df = pd.concat([df, cur_df], ignore_index=True)
		t3 = time.time()
		# print("===================== retry: ", retry)
		# print(f"====代码运行时间: {t2 - t1} 秒")
		# print(f"====代码运行时间: {t3 - t2} 秒")
		# print("====",len(df),"=====")

		prev_date_string = get_previous_date(end_date_string, period=1)
		return quick_cursor(code, prev_date_string, df, retry=retry-1)
	else:
		return df

In [ ]:

def get_minutes_info(codes, end_date_string, min_count = 1500):
	minutes_df = pd.DataFrame()

	# DCE_future_price_collection.create_index([("date", pymongo.DESCENDING),("clock", pymongo.DESCENDING)])
	print(codes)
	for code in codes:
		print("cur code: ",code)
		cur_df = pd.DataFrame()
		# cur_df = quick_cursor2(code, end_date_string)
		cur_df = quick_cursor(code, end_date_string, cur_df, min_count = min_count, retry=5)

		if len(cur_df)>= min_count:
			t1= time.time()
			cur_df = cur_df.sort_values(by=['date','clock'], ascending=True)
			minutes_df = pd.concat([minutes_df, cur_df], ignore_index=True)
			t2 = time.time()
			# print(f"==*********==代码运行时间: {t2 - t1} 秒")
	
	return minutes_df

In [ ]:
def check_upper_ma(prices, realtime_prices, period, if_minute = False):
	unique_codes = prices['future_code'].unique()
	newest_prices = realtime_prices[["future_code", "newest_price"]]
	ma_selected_code = []
	for code in unique_codes:
		sub_df = prices[prices['future_code'] == code]
		if if_minute:
			close_prices = sub_df["newest_price"]
		else:
			close_prices = sub_df["close"]
		# print(close_prices)
		ma_value = TT.MA(close_prices, period)[-1]

		temp = newest_prices.loc[newest_prices["future_code"] == code,"newest_price"]
		if len(temp) >0:
			if ma_value < temp.item():
				ma_selected_code.append(code)	

	return ma_selected_code

In [ ]:
def check_daily_kdj(daily_prices, codes, tor = 1):
	selected_code = []
	for code in codes:
		sub_df = daily_prices[daily_prices["future_code"] == code]
		close, high, low = sub_df['close'], sub_df['high'], sub_df['low']
		K, D, J = KDJ(close, high, low)

		# print("=================")
		# print(K)
		# print(D)
		if len(K) >= tor and len(D) >= tor:
			_, res = check_cross(K, D)

			if res == "gold":
				selected_code.append(code)

	return selected_code

In [ ]:
def check_hourly_kdj(daily_prices, codes, tor = 2):
	
	daily_prices['hour'] = daily_prices['clock'] // 10000  # 将 230000 转换为 23，210000 转换为 21 等

	# 根据 future_code 和每小时进行分组，并聚合
	hourly_prices = daily_prices.groupby(['future_code', 'date', 'hour']).agg({
		'newest_price': ['last', 'max', 'min'],
		'future_code': 'first',
		'date': 'first',
		'clock': 'last'
	}).reset_index()

	# 调整列名
	hourly_prices.columns = ['future_code', 'date', 'hour', 'close', 'high', 'low', 'future_code_', 'date_', 'clock_']
	hourly_prices = hourly_prices[['future_code', 'date', 'clock_', 'close', 'high', 'low']]
	hourly_prices.columns = ['future_code', 'date', 'clock', 'close', 'high', 'low']

	hourly_prices = hourly_prices.sort_values(by=["future_code","date","clock"], ascending=[True, True, True])
	# hourly_prices.to_csv("./hourly_df.csv", index=False, encoding="gbk")	

	selected_code = []
	for code in codes:
		sub_df = hourly_prices[hourly_prices["future_code"] == code]
		close, high, low = sub_df['close'], sub_df['high'], sub_df['low']
		K, D, J = KDJ(close, high, low)

		if len(K) >= tor and len(D) >= tor:
			_, res = check_cross(K, D)

			if res == "gold":
				selected_code.append(code)

	return selected_code

In [ ]:

"""
	Here, 'end_date_string' typically represents today. 
	However, 'end_date' should be adjusted to be one day ahead ('end_date_ahead') for easier calculations, 
	especially when computing indicators such as MA and KDJ.
"""
def get_name_list(end_date_string, tt, period = 20, period_minute = 20, threshold = 0.05):
	end_date = datetime.datetime.strptime(end_date_string, "%Y-%m-%d")
	end_date_ahead = end_date - datetime.timedelta(days=1)
	end_date_ahead_string = end_date_ahead.strftime('%Y-%m-%d')  # 格式为 YYYY-MM-DD

	if is_valid_date(end_date_string):

		"""
			C1: 正基差过滤
		"""
		t1 = time.time()
		# get namelist by basis of percentage
		print(end_date_ahead_string)
		basis_future = pd.DataFrame([i for i in DCE_commodity_price_collection.find(
						# {"date": end_date_string}
						{"date": end_date_ahead_string}
					).sort("main_future_code", pymongo.ASCENDING) ])
		# print(basis_future)
		# print(basis_future)
		basis_future = basis_future.sort_values(by='main_cf_basis_percent', ascending=False)

		t2 = time.time()

		# Find the main contract with +5% positive basis 
		codes = basis_future[basis_future["main_cf_basis_percent"]>=threshold]["main_future_code"].tolist()
		codes = list(set(codes))
		# ok---------------------------------------------------------------------------------------------------
		"""
			C2: 价格高于20日均线，且日KDJ交金叉
		"""
		# A larger date range (2*period) to ensure there's no data shortage 
		# due to non-trading days on weekends (Saturday and Sunday).
		start_date_string = get_previous_date(date_string = end_date_string, period=period*2+1) 
		dates_list = generate_dates(start_date_string, end_date_string)

		# get the price information of main contract, including high,low, open,close
		daily_prices = get_daily_info(codes, dates_list, period) 
		# print(daily_prices) #['_id', 'date', 'future_code', 'close', 'open', 'high', 'low', 'deal', 'categoryID'],
		
		t3 = time.time()
		# print(f'xxxxx')
		# print(end_date_string)
		realtime_prices = get_realtime_price(end_date_string, tt, codes)
		# realtime_prices.to_csv("./realtime_prices.csv", index=False, encoding="gbk")

		ma_selected_codes = check_upper_ma(daily_prices, realtime_prices, period)
		print(f'xxma_selectedxxx')
		print(ma_selected_codes)
		print(f'xxxxx')
		t4 = time.time()

		kdj_selected_codes = check_daily_kdj(daily_prices, ma_selected_codes)
		print(f'xxkdj_selected_codesxx')
		print(kdj_selected_codes)
		print(f'xxxx')
		# kdj_selected_codes = check_daily_kdj(daily_prices, codes)
		# print(kdj_selected_codes)

		"""
			C3: 价格高于60分钟-20均线，且60分钟-KDJ交金叉
		"""
		t5 = time.time()

		# for fast verification
		if os.path.isfile("./minutes_df.csv"):
			minutes_df = pd.read_csv("./minutes_df.csv", encoding="gbk")
		else:
			minutes_df = get_minutes_info(codes, end_date_string)
			# minutes_df.to_csv("./minutes_df.csv", index=False, encoding="gbk")	
			print(len(minutes_df))

		t6 = time.time()

		hourly_prices = minutes_df.groupby('future_code').apply(\
			lambda group: group.iloc[::-60]).reset_index(drop=True)

		ma_selected_codes_hourly = check_upper_ma(hourly_prices, realtime_prices, period_minute, if_minute=True)
		# print(ma_selected_codes_hourly)

		kdj_selected_codes_hourly = check_hourly_kdj(minutes_df, ma_selected_codes_hourly)
		# print(kdj_selected_codes_hourly)
		t7 = time.time()

		# 创建空的 DataFrame
		final_codes = list(set(kdj_selected_codes) & set(kdj_selected_codes_hourly))
		final_results = pd.DataFrame(columns=['评估指标','评估结果'])
		final_results.loc[0] = ["正基差", codes]
		final_results.loc[1] = ["价格高于20日均线，且日KDJ交金叉", kdj_selected_codes]
		final_results.loc[2] = ["价格高于60分钟-20均线，且60分钟-KDJ交金叉", kdj_selected_codes]
		final_results.loc[3] = ["综合结果", final_codes]
		final_results.to_csv("./final_results.csv", index=False, encoding="gbk")

		# print(f"代码运行时间: {t2 - t1} 秒")
		# print(f"代码运行时间: {t3 - t2} 秒")
		# print(f"代码运行时间: {t4 - t3} 秒")
		# print(f"代码运行时间: {t5 - t4} 秒")
		# print(f"代码运行时间: {t6 - t5} 秒")
		# print(f"代码运行时间: {t7 - t6} 秒")
		# print("正基差： ",codes)
		# print("ma_selected_codes ",ma_selected_codes)
		# print("kdj_selected_codes ",kdj_selected_codes)
		# print("ma_selected_codes_hourly ",ma_selected_codes_hourly)
		# print("kdj_selected_codes_hourly ",kdj_selected_codes_hourly)
		# print("final_codes ", final_codes)
		# print(final_results)
		print(final_results)
		return final_results
	else:
		print("错误日期信息")
		return None

In [79]:
date = [datetime.date(2023,9,12)+datetime.timedelta(days=i) for i in [1,2,3,7,8,9,10]]
timepoint = [93500,94500,95500,
             100500,101500,102500,103500,104500,105500,
             110500,111500,112500] + [133500, 134500,135500,
                                      140500,141500,142500,143500,144500,145500]
resultpd = pd.DataFrame()
resultpd['date']=[]
resultpd['time']=[]
resultpd['code']=[]

eval_resultpd = pd.DataFrame()
eval_resultpd['date']=[]
eval_resultpd['time']=[]
eval_resultpd['code']=[]
eval_resultpd['t20']=[]
eval_resultpd['t60']=[]
eval_resultpd['td']=[]
eval_resultpd['t']=[]


In [ ]:
for d in date:
    for t in timepoint:
        r = get_name_list(str(d),t)
        if r is None:
            continue
        else:
            l = r.iat[2,1]
            if len(l)==0:
                continue
            else:
                resultpd = pd.concat([resultpd, pd.DataFrame({'date': d, 'time': t,'code':l})], ignore_index=True)
                # resultpd = resultpd.append({'date': d, 'time': t,'code':l}, ignore_index=True)
if len(resultpd)>0:
    resultpd.to_csv('resultpd.csv', encoding='utf-8', index=False)

In [78]:
import pymongo

DCE_DB_NAME = "DCEdb"
DCE_FUTURE_PRICE_COLLECTION_NAME = "DCEFuturePriceDb"
COSMOS_CONNECTION_STRING = 'mongodb://lei:WMNi1Hh4vyOVG4V3OLcyaoZwOOl7YVHUUiKFeuGKbiS9rytxi2qd2VpnekCYj8IVrN6AFVHvLBUxACDb7vuFnA==@lei.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@lei@'
client = pymongo.MongoClient(COSMOS_CONNECTION_STRING)
DCEdb = client[DCE_DB_NAME]
DCE_future_price_collection = DCEdb[DCE_FUTURE_PRICE_COLLECTION_NAME]
def get_price(date,tt,code):
	t = DCE_future_price_collection.find(
							{"$and":[
								{'date':date},
								{'clock':{"$lt":int(tt)}},
								{'future_code':code}]}
							).sort("update_time", pymongo.DESCENDING).limit(1)
	t20 = DCE_future_price_collection.find(
							{"$and":[
								{'date':date},
								{'clock':{"$lt":int(tt + 2000) if tt+2000<113000 else int(tt+20000+2000)}},
								{'future_code':code}]}
							).sort("update_time", pymongo.DESCENDING).limit(1)
	t60 = DCE_future_price_collection.find(
							{"$and":[
								{'date':date},
								{'clock':{"$lt":int(tt + 10000) if tt+10000<113000 else int(tt+20000+10000)}},
								{'future_code':code}]}
							).sort("update_time", pymongo.DESCENDING).limit(1)
	td = DCE_future_price_collection.find(
							{"$and":[
								{'date':date},
								{'clock':{"$lt":int(170000)}},
								{'future_code':code}]}
							).sort("update_time", pymongo.DESCENDING).limit(1)
	return pd.DataFrame(t).iloc[0]['newest_price'],pd.DataFrame(t20).iloc[0]['newest_price'], pd.DataFrame(t60).iloc[0]['newest_price'], pd.DataFrame(td).iloc[0]['newest_price']

In [85]:
resultpd = pd.read_csv('resultpd.csv')

for index, row in resultpd.iterrows():
    print(f'searching {row[0]} {row[1]} {row[2]}')
    t_,t20,t60,td = get_price(row[0],int(row[1]),row[2])
    print(pd.DataFrame({'date': row[0], 'time': row[1],'code':l,'t':row[2],'t20':t20,'t60':t60,'td':td}))
    eval_resultpd = pd.concat([eval_resultpd, pd.DataFrame({'date': [row[0]], 'time': [row[1]],'code':[row[2]],'t':t_,'t20':t20,'t60':t60,'td':td})], ignore_index=True)
if len(eval_resultpd)>0:
    eval_resultpd.to_csv('eval_resultpd.csv', encoding='utf-8', index=False)

searching 2023-09-14 93500.0 I2401
         date     time    code      t    t20    t60     td
0  2023-09-14  93500.0  FU2401  I2401  858.0  855.0  863.5
searching 2023-09-14 94500.0 I2401
         date     time    code      t    t20    t60     td
0  2023-09-14  94500.0  FU2401  I2401  857.5  856.0  863.5
searching 2023-09-14 95500.0 I2401
         date     time    code      t    t20    t60     td
0  2023-09-14  95500.0  FU2401  I2401  857.5  856.0  863.5
searching 2023-09-14 100500.0 I2401
         date      time    code      t    t20    t60     td
0  2023-09-14  100500.0  FU2401  I2401  855.5  858.0  863.5
searching 2023-09-14 101500.0 I2401
         date      time    code      t    t20    t60     td
0  2023-09-14  101500.0  FU2401  I2401  855.0  861.0  863.5
searching 2023-09-14 102500.0 I2401
         date      time    code      t    t20    t60     td
0  2023-09-14  102500.0  FU2401  I2401  856.0  866.0  863.5
searching 2023-09-14 103500.0 I2401
         date      time    code      